In [ ]:
import pandas as pd
import numpy as np
import os
import random
import re
import matplotlib.pyplot as plt

In [ ]:
os.getcwd()

In [ ]:
data = pd.read_csv("abelin_mono_pep.csv")
data= data[["peptide","HLA"]]

In [ ]:
data["len"] = data["peptide"].map(len)
data = data[data["len"]>=13]
data = data[data["len"]<=21]
data = data.reset_index(drop=True)

In [ ]:
data["len"].value_counts()

In [ ]:
data["label"] = 1

In [ ]:
# generated negative examples
def shuffle_str(s):
    random.seed(20230304)
    str_list = list(s)
    random.shuffle(str_list)
    return ''.join(str_list)

In [ ]:
neg_data = pd.DataFrame()
neg_data["peptide"] = data["peptide"].map(shuffle_str)

In [ ]:
neg_data["HLA"] = data["HLA"]
neg_data["len"] = neg_data["peptide"].map(len)
neg_data["label"] = 0

In [ ]:
neg_data

In [ ]:
pseudo_seq = pd.read_table("pseudosequence_MHCII.txt",header = None, names = ["HLA","sequence"])

In [ ]:
mono_pep = pd.concat([data,neg_data])
mono_pep = mono_pep.reset_index(drop=True)
mono_pep = pd.merge(mono_pep,pseudo_seq)

In [ ]:
mono_pep

In [ ]:
mono_pep.to_csv("final_mono_pep.csv")

# mutiple allele

In [ ]:
# mutiple allele
mutile_pep = pd.read_csv("abelin_multiple_pep.csv")
mutiple_pep = mutile_pep.dropna()
mutiple_pep = mutile_pep.drop_duplicates()
mutiple_allele = pd.read_csv("abelin_mutiple_allele.csv")

In [ ]:
mutiple_pep["len"] = mutiple_pep["peptide"].map(len)
mutiple_pep = mutiple_pep[mutiple_pep["len"]>=13]
mutiple_pep = mutiple_pep[mutiple_pep["len"]<=21]
mutiple_pep = mutiple_pep.reset_index(drop=True)
mutiple_pep["label"] = 1

In [ ]:
mutiple_pep["len"].value_counts()

In [ ]:
mutiple_pep

In [ ]:
neg_mutiple_pep = pd.DataFrame()
neg_mutiple_pep["peptide"] = mutiple_pep["peptide"].map(shuffle_str)
neg_mutiple_pep["Sample Name"] = mutiple_pep["Sample Name"]
neg_mutiple_pep["len"] = neg_mutiple_pep["peptide"].map(len)
neg_mutiple_pep["label"] = 0

In [ ]:
neg_mutiple_pep

In [ ]:
Mutiple_pep = pd.concat([mutiple_pep,neg_mutiple_pep])
Mutiple_pep = Mutiple_pep.reset_index(drop=True)

In [ ]:
Mutiple_pep = pd.merge(Mutiple_pep,mutiple_allele)

In [ ]:
Mutiple_pep = pd.merge(Mutiple_pep,pseudo_seq)

In [ ]:
Mutiple_pep

In [ ]:
Mutiple_pep.to_csv("final_multiple_pep.csv")

In [ ]:
#change allele to suit netMHCIIpan and mixMHC2pred
mono_pep["net_HLA"] = 

In [ ]:
mono_pep["HLA"].value_counts()

In [ ]:
def hla2net(x):
    if len(x)<10:
        x = "HLA-" + x
        x = re.sub("_","*",x)
        x = x[0:11] + ":" + x[11:]
    else:
        
        x = re.sub("-","/",x)
        x= x[0:8]+"*" + x[8:10] + ":" + x[10:17] + "*" + x[17:19] +":" +x[19:]
        x = x[4:]
        x = "HLA-" + x
    return x

In [ ]:
mono_pep["net_HLA"] = mono_pep["HLA"].map(hla2net)

In [ ]:
def hla2mix(x):
    if len(x)<10:
        x = x[0:7] + "_" + x[7:9]
    else:
        x = re.sub("-","",x)
        x = x[3:]
        x= x[0:4]+"_" + x[4:6] + "_" + x[6:8] + "__" + x[8:12] +"_" +x[12:14]+"_" + x[14:]
    return x
mono_pep["mix_HLA"] = mono_pep["HLA"].map(hla2mix)

In [ ]:
mono_pep["mix_HLA"].value_counts()

In [ ]:
mono_pep.to_csv("final_mono_pep.csv")

In [ ]:
Mutiple_pep["net_HLA"] = Mutiple_pep["HLA"].map(hla2net)
Mutiple_pep["mix_HLA"] = Mutiple_pep["HLA"].map(hla2mix)

In [ ]:
Mutiple_pep.to_csv("final_multiple_pep.csv")

In [ ]:
Mutiple_pep

#result

In [ ]:
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
from sklearn.metrics import auc,roc_curve,precision_recall_curve
from sklearn.metrics import confusion_matrix
import math

In [ ]:
BA_mono = pd.read_csv("result/BA_Abelin_mono_result.csv")
BA_mono.pop("pep_blosum")
BA_mono.pop("MHC_blosum")
BA_mono.pop("pep_blosum")
BA_mono.pop("MHC_blosum")

In [ ]:
BA_fpr,BA_tpr,_ = roc_curve(BA_mono["label"],1-BA_mono["Rank"],pos_label=1)
BA_roc = auc(BA_fpr,BA_tpr)
BA_roc

In [ ]:
#multiple allele
BA_multiple = pd.read_csv("result/BA_Abelin_multiple_result.csv")
BA_multiple.pop("MHC_blosum")
BA_multiple.pop("pep_blosum")
BA_multiple

In [ ]:
label = []
Rank = []
for s in BA_multiple["Sample Name"].unique():
    BA_multipe_s = BA_multiple[BA_multiple["Sample Name"] == s]
    for p in BA_multipe_s["peptide"].unique():
        BA_multipe_p = BA_multipe_s[BA_multipe_s["peptide"] == p]
        label.append(BA_multipe_p["label"][BA_multipe_p["Rank"].idxmin()])
        Rank.append(BA_multipe_p["Rank"][BA_multipe_p["Rank"].idxmin()])
BA_multiple_result = pd.DataFrame({"label":label,"Rank":Rank})

In [ ]:
BA_mono_result = BA_mono[["label","Rank"]]
BA_result = pd.concat([BA_mono_result,BA_multiple_result])
BA_fpr,BA_tpr,_ = roc_curve(BA_result["label"],1-BA_result["Rank"],pos_label=1)
BA_roc = auc(BA_fpr,BA_tpr)
BA_roc

# mixMHC2pred

In [ ]:
# show result in different length and allele

In [ ]:
mix_mono = pd.read_csv("result/mixMHC2pred_Abelin_mono_result.csv")
mix_mono = pd.merge(mix_mono[["Peptide","%Rank_best","BestAllele"]],mono_pep[["peptide","label","mix_HLA"]],left_on=["Peptide","BestAllele"],right_on=["peptide","mix_HLA"]).drop_duplicates()

In [ ]:
mix_fpr,mix_tpr,_ = roc_curve(mix_mono["label"],1-mix_mono["%Rank_best"],pos_label=1)
mix_mono_roc = auc(mix_fpr,mix_tpr)
mix_mono_roc

In [ ]:
#multiple
mix_multi = pd.read_csv("result/mixMHC2pred_Abelin_multiple_result.csv")
mix_multi = pd.merge(mix_multi[["Peptide","%Rank_best","BestAllele"]],Mutiple_pep[["peptide","Sample Name","label","mix_HLA"]],left_on=["Peptide","BestAllele"],right_on=["peptide","mix_HLA"]).drop_duplicates()

In [ ]:
label = []
Rank = []
for s in mix_multi["Sample Name"].unique():
    mix_multi_s = mix_multi[mix_multi["Sample Name"] == s]
    for p in mix_multi_s["peptide"].unique():
        mix_multi_p = mix_multi_s[mix_multi_s["peptide"] == p]
        label.append(mix_multi_p["label"][mix_multi_p["%Rank_best"].idxmin()])
        Rank.append(mix_multi_p["%Rank_best"][mix_multi_p["%Rank_best"].idxmin()])
mix_multi_result = pd.DataFrame({"label":label,"%Rank_best":Rank})

In [ ]:
mix_mono_result = mix_mono[["label","%Rank_best"]]
mix_result = pd.concat([mix_mono_result,mix_multi_result])
mix_fpr,mix_tpr,_ = roc_curve(mix_result["label"],1-mix_result["%Rank_best"],pos_label=1)
mix_roc = auc(mix_fpr,mix_tpr)
mix_roc

# NetMHCIIpan

In [ ]:
net_mono=pd.read_csv("netMHCIIpan/netMHCIIpan_mono_result.csv")
net_mono = pd.merge(net_mono[["peptide","percentile_rank","allele"]],mono_pep[["peptide","label","net_HLA"]],left_on=["peptide","allele"],right_on=["peptide","net_HLA"]).drop_duplicates()

In [ ]:
net_fpr,net_tpr,_ = roc_curve(net_mono["label"],1-net_mono["percentile_rank"],pos_label=1)
net_mono_roc = auc(net_fpr,net_tpr)
net_mono_roc

In [ ]:
net_multi = pd.read_csv("netMHCIIpan/netMHCIIpan_multiple_result.csv")
net_multi = pd.merge(net_multi[["peptide","percentile_rank","allele"]],Mutiple_pep[["peptide","Sample Name","label","net_HLA"]],left_on=["peptide","allele"],right_on=["peptide","net_HLA"]).drop_duplicates()

In [ ]:
label = []
Rank = []
for s in net_multi["Sample Name"].unique():
    net_multi_s = net_multi[net_multi["Sample Name"] == s]
    for p in net_multi_s["peptide"].unique():
        net_multi_p = net_multi_s[net_multi_s["peptide"] == p]
        label.append(net_multi_p["label"][net_multi_p["percentile_rank"].idxmin()])
        Rank.append(net_multi_p["percentile_rank"][net_multi_p["percentile_rank"].idxmin()])
net_multi_result = pd.DataFrame({"label":label,"percentile_rank":Rank})

In [ ]:
net_mono_result = net_mono[["label","percentile_rank"]]
net_result = pd.concat([net_mono_result,net_multi_result])
net_fpr,net_tpr,_ = roc_curve(net_result["label"],1-net_result["percentile_rank"],pos_label=1)
net_roc = auc(net_fpr,net_tpr)
net_roc

# Figure 1B

In [ ]:
Automate = pd.read_csv("data/Automated_benchmark.csv")
BA_result = pd.read_csv("data/BA_Automated_prediction_result.csv")
BA_fpr,BA_tpr,_ = roc_curve(BA_result["label"],BA_result["prediction"],pos_label=1)
BA_roc = auc(BA_fpr,BA_tpr)
netMHC_result = pd.read_csv("data/netMHCIIpan_Auto_filter_ba_result.csv")
netMHC_result = netMHC_result[["allele","percentile_rank","peptide","ic50"]]
Automate = Automate[["allele","allele1","peptide_sequence","measurement_value"]]
netMHC_result = pd.merge(netMHC_result,Automate,left_on=["allele","peptide"],right_on=["allele","peptide_sequence"]).drop_duplicates()
BA_allele = pd.read_csv("data/BA_allele.csv",header=None,names=["allele"])
bool = netMHC_result["allele1"].map(lambda x : x in BA_allele["allele"].values)
netMHC_result = netMHC_result[bool]
net_fpr,net_tpr,_ = roc_curve(netMHC_result["measurement_value"],1-netMHC_result["percentile_rank"],pos_label=1)
net_roc = auc(net_fpr,net_tpr)
mix_result = pd.read_csv("data/mixMHC2pred_Auto_filter_ba_result.csv")
mix_data = pd.read_csv("data/mixMHC2pred_Automated.csv")
mix_result = pd.merge(mix_result[["Peptide","BestAllele","%Rank_best"]],mix_data[["Description","new_allele","label","Allele Name1"]],left_on =["Peptide","BestAllele"],right_on=["Description","new_allele"]).drop_duplicates()
mix_result = mix_result[mix_result["Allele Name1"].isin(BA_allele["allele"])]
mix_fpr,mix_tpr,_ = roc_curve(mix_result["label"],1-mix_result["%Rank_best"],pos_label=1)
mix_roc = auc(mix_fpr,mix_tpr)
cmap = plt.get_cmap('Set1')
colors = [mpl.colors.rgb2hex(cmap(i)[:3]) for i in range(3)]
fig,ax = plt.subplots(figsize = (8,8))
lw = 2
plt.plot(BA_fpr,BA_tpr,lw=lw,color = colors[0],label="BA_model (AUC = %0.4f)" % BA_roc,)
plt.plot(net_fpr,net_tpr,lw=lw,color = colors[1],label="NetMHCIIpan_BA (AUC = %0.4f)" % net_roc,)
plt.plot(mix_fpr,mix_tpr,lw=lw,color = colors[2],label="mixMHC2pred (AUC = %0.4f)" % mix_roc,)
plt.plot([0, 1], [0, 1], color="navy", lw=lw, linestyle="--")
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel("False Positive Rate",fontsize = 14)
plt.ylabel("True Positive Rate",fontsize = 14)
plt.title("")
plt.legend(loc="lower right",fontsize = 12)
plt.show()

In [ ]:
Automate = pd.read_csv("data/Automated_benchmark.csv")
BA_result = pd.read_csv("data/BA_Automated_prediction_rank_result.csv")

In [ ]:
BA_result

In [ ]:
BA_result["len"] = BA_result["pep"].map(len)
for i in BA_result["len"].unique():
    BA_result_l = BA_result[BA_result["len"] == i]
    BA_fpr,BA_tpr,_ = roc_curve(BA_result_l["label"],BA_result_l["prediction"],pos_label=1)
    BA_roc = auc(BA_fpr,BA_tpr)
    print("Length:{},ROC:{}".format(i,BA_roc))

In [ ]:
Allele_Name = []
ROC = []
for i in BA_result["Allele Name1"].unique():
    BA_result_l = BA_result[BA_result["Allele Name1"] == i]
    BA_fpr,BA_tpr,_ = roc_curve(BA_result_l["label"],BA_result_l["prediction"],pos_label=1)
    BA_roc = auc(BA_fpr,BA_tpr)
    Allele_Name.append(i)
    ROC.append(BA_roc)
    print("Allele Name:{},ROC:{}".format(i,BA_roc))
BA_allele_result = pd.DataFrame({"Allele Name":Allele_Name,"BA_model":ROC})

In [ ]:
BA_allele_result

In [ ]:
netMHC_result["len"] = netMHC_result["peptide"].map(len)

In [ ]:
def split_roc(df,col,lab,pre):
    Allele_Name = []
    ROC = []
    for i in df[col].unique():
        df_l = df[df[col] == i]
        df_fpr,df_tpr,_ = roc_curve(df_l[lab],1-df_l[pre],pos_label=1)
        df_roc = auc(df_fpr,df_tpr)
        Allele_Name.append(i)
        ROC.append(df_roc)
    allele_result = pd.DataFrame({"Allele Name":Allele_Name,"roc":ROC})
    return allele_result

In [ ]:
split_roc(netMHC_result,"len","measurement_value","percentile_rank")

In [ ]:
net_allele_roc = split_roc(netMHC_result,"allele1","measurement_value","percentile_rank")

In [ ]:
mix_result["len"] = mix_result["Peptide"].map(len)

In [ ]:
mix_result

In [ ]:
split_roc(mix_result,"len","label","%Rank_best")

In [ ]:
mix_allele_roc = split_roc(mix_result,"Allele Name1","label","%Rank_best")

In [ ]:
df1 = pd.merge(BA_allele_result,net_allele_roc)
df1 = df1.rename(columns={"roc":"netMHCIIpan"})
df2 = pd.merge(df1,mix_allele_roc)
df2 = df2.rename(columns={"roc":"mixMHC2pred"})

In [ ]:
df2.to_csv("fig1b_allele.csv")